# Explolation06
### 가사.txt를 rnn학습시켜 멋진 가사를 만들어보자.

## 필요한 모듈 가져오기

In [23]:
import glob
import tensorflow as tf
import os, re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
print(matplotlib.__version__)


3.2.2


## 가사데이터들을 가져오기

In [25]:
txt_list = glob.glob('./lyrics/*')

## 가져온 데이터들을 리스트형식으로 저장해주기.

In [10]:
raw_corpus = []


for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:9])

데이터 크기: 187088
Examples:
 ['I saw you walking by his side heard you whisper all those lies', "And I couldn't keep from crying", 'You sang him love songs tenderly that should have been for you and me', "And I couldn't keep from crying", 'I saw his eyes drinking your charms while he held you in his arms', 'Him with all his wedding ways rules your heart now in my place', "I stood and watched him steal a kiss from two lips I know I'll miss", "And I couldn't keep from crying I saw his eyes drinking your charms... I love that hair, long an' black", "Hangin' down to the middle of your back"]


In [11]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   
  
    if idx >30:break
    print(sentence)

I saw you walking by his side heard you whisper all those lies
And I couldn't keep from crying
You sang him love songs tenderly that should have been for you and me
And I couldn't keep from crying
I saw his eyes drinking your charms while he held you in his arms
Him with all his wedding ways rules your heart now in my place
I stood and watched him steal a kiss from two lips I know I'll miss
And I couldn't keep from crying I saw his eyes drinking your charms... I love that hair, long an' black
Hangin' down to the middle of your back
Don't cut it off whatever you do
I need it to run my fingers through 'Cause you're my baby, uh huh uh, you're my sugar
Don't mean maybe, you're my, baby Got me a dollar that I saved
Saved it up for a rainy day
Everybody's callin' for bills that's due
But they don't catch me, I'll spend it on you 'Cause you're my baby, uh huh uh, you're my sugar
Don't mean maybe, you're my, baby Got me a guitar, got a six strings
And a picker to make 'em ring
Every string's g

## 잘 가져와진 모습이다. 이제 필요없는 문자열을 삭제해주자
## 다음의 코드는 필요없는 문자열을 삭제해주는 함수이다.

In [12]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>' 
    return sentence


print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [13]:
corpus = []

for sentence in raw_corpus:
   
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        

corpus[:10]

['<start> i saw you walking by his side heard you whisper all those lies <end>',
 '<start> and i couldn t keep from crying <end>',
 '<start> you sang him love songs tenderly that should have been for you and me <end>',
 '<start> and i couldn t keep from crying <end>',
 '<start> i saw his eyes drinking your charms while he held you in his arms <end>',
 '<start> him with all his wedding ways rules your heart now in my place <end>',
 '<start> i stood and watched him steal a kiss from two lips i know i ll miss <end>',
 '<start> and i couldn t keep from crying i saw his eyes drinking your charms . . . i love that hair , long an black <end>',
 '<start> hangin down to the middle of your back <end>',
 '<start> don t cut it off whatever you do <end>']

## 잘 삭제된 모습이다. 이제 이 데이터를 토큰화하는 함수로 토큰화해보자.

In [14]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    5  475 ...    0    0    0]
 [   2    8    5 ...    0    0    0]
 [   2    7 1731 ...    0    0    0]
 ...
 [   2    5   22 ...    0    0    0]
 [   2    5   22 ...    0    0    0]
 [   2    5   22 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7feb567f18d0>


## 잘 토큰화가 된 모습이다. 필자는 단어장을 12000으로 해보았다. 보통 7000~8000개를 한다고 한다.
## 그리고 나는 maxlen값을 쓰지않았다. 학습시간이 클것같긴하지만 나는 문장의 수가 많으면 많을수록 좋다고 가정을 했다.

In [15]:
print(tensor[:3, :10])

[[   2    5  475    7  775  122  105  300  297    7]
 [   2    8    5  601   15  129   74  878    3    0]
 [   2    7 1731  142   33  903 4376   17  198   76]]


In [16]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


## 토큰화도 잘 된 모습이다. 이제 발리데이션 로스를 확인하기위해 트레인과 테스트 데이터로 분리해주자.

In [17]:
from sklearn.model_selection import train_test_split
src_input = tensor[:, :-1]  

tgt_input = tensor[:, 1:]   
(enc_train, enc_val, dec_train, dec_val) = train_test_split(src_input,tgt_input,test_size=0.2,shuffle=True,random_state=500)

print(src_input[0])
print(tgt_input[0])

[   2    5  475    7  775  122  105  300  297    7 1710   24  423  709
    3    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

## 여기서 필자는 배치사이즈를 128로 조정해주었다. 256이상부터는 아웃오브메모리 즉, 메모리가 부족한 상태가 뜨길래 128로 해 주었다. 아까 maxlen의 값을 안해줘서 그런것같다.
## 그래도 배치사이즈를 조정해 오류를 없앴다.


In [18]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 128
steps_per_epoch = len(src_input) // BATCH_SIZE


VOCAB_SIZE = tokenizer.num_words + 1   
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset_test = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_test = dataset_test.shuffle(BUFFER_SIZE)
dataset_test = dataset_test.batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(128, 346), dtype=tf.int32, name=None), TensorSpec(shape=(128, 346), dtype=tf.int32, name=None))>

## 이제 데이터를 학습시키기 위해 모델링을 해주자. 필자는 임베딩1층, rnn2층, 리니어1층으로 모델링을 해주었으며, 임베딩 사이즈는 512, 은닉층의 뉴런갯수는 1024개로 해주었다.
## 아무래도 뉴런의 갯수가 많아지면 많아 질수록 학습시간이 오래걸릴것같다. 하지만 그만큼 학습률은 좋을듯하다. 다른 단점으로는 과적합이 잘 일어날것같은데 epoch수를 작게해서 모델을 돌려보자.

In [19]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

## 샘플로 한 배치만 데이터를 모델에 넣어보았다.

In [20]:
for src_sample, tgt_sample in dataset.take(1): break


model(src_sample)

<tf.Tensor: shape=(128, 346, 12001), dtype=float32, numpy=
array([[[ 1.59427596e-04, -1.13546484e-04, -1.34676869e-04, ...,
          1.19838805e-04,  1.27121268e-04, -2.10279672e-04],
        [ 3.90956120e-04, -2.11121387e-05, -2.25890399e-04, ...,
          1.93930435e-04, -1.35101873e-04, -2.61748180e-04],
        [ 5.03247429e-04,  4.13110189e-04, -3.06453148e-04, ...,
          7.03537138e-04, -5.65044211e-05, -3.17306491e-04],
        ...,
        [ 4.58456983e-04, -3.47380713e-03, -4.75838920e-03, ...,
          1.03367587e-04, -1.59916433e-03, -2.38659602e-04],
        [ 4.58457042e-04, -3.47380619e-03, -4.75838827e-03, ...,
          1.03367005e-04, -1.59916386e-03, -2.38659370e-04],
        [ 4.58456809e-04, -3.47380480e-03, -4.75838874e-03, ...,
          1.03367005e-04, -1.59916386e-03, -2.38659370e-04]],

       [[ 1.59427596e-04, -1.13546484e-04, -1.34676869e-04, ...,
          1.19838805e-04,  1.27121268e-04, -2.10279672e-04],
        [ 4.36619535e-04, -2.92623066e-04, -

## 층을 잘 쌓았나 확인해본뒤 본격적으로 학습시켜보자.

In [21]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  6144512   
                                                                 
 lstm (LSTM)                 multiple                  6295552   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  12301025  
                                                                 
Total params: 33,133,793
Trainable params: 33,133,793
Non-trainable params: 0
_________________________________________________________________


In [22]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=5, validation_data =dataset_test)

Epoch 1/5
1098/1098 [==============================] - 2558s 2s/step - loss: 0.1892 - val_loss: 0.1417
Epoch 2/5
1098/1098 [==============================] - 2577s 2s/step - loss: 0.1347 - val_loss: 0.1307
Epoch 3/5
1098/1098 [==============================] - 2570s 2s/step - loss: 0.1244 - val_loss: 0.1234
Epoch 4/5
1098/1098 [==============================] - 2571s 2s/step - loss: 0.1152 - val_loss: 0.1173
Epoch 5/5
1098/1098 [==============================] - 2591s 2s/step - loss: 0.1056 - val_loss: 0.1124


## 학습결과 과적합은 일어나보이지않은 모습이며 loss값 발리데이션 loss값이 다 낮은 모습이다. 이제 가사를 뽑아보자.

In [27]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
  
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

  
    while True:
        
        predict = model(test_tensor) 
        
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
   
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [28]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he s a <unk> , he s a mess , he s a monster <end> '

## ㅋㅋㅋ뭔가 랩가사를 만들어준것같다. 그는 엉망이고, 그는 괴물이고, 어느 여인이 남자를 원망하는 가사인것같다. 연애할때 남자가 잘 못해주었나보다. 

# 이번에는 가사데이터를 통해 가사를 만들어봤는데, rnn의 모델중 lstm은 엄청 복잡한 모델인것같다. 그래서 학습시간도 긴것같다. 하지만 그만큼 성능은 좋은듯 보인다. 이것으로 Explolation06  과제를 마친다.